In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import random
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')
data_read = pd.read_csv('/content/drive/My Drive/LHL_Final_Project/Written_Reviews/Beer_Recommendations_EDA_V01.csv')

Mounted at /content/drive


In [4]:
!pip install streamlit
!pip install pyngrok

     |████████████████████████████████| 8.0 MB 9.0 MB/s 
     |████████████████████████████████| 75 kB 213 kB/s 
     |████████████████████████████████| 170 kB 61.6 MB/s 
     |████████████████████████████████| 111 kB 73.4 MB/s 
     |████████████████████████████████| 4.3 MB 63.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 123 kB 61.4 MB/s 
     |████████████████████████████████| 787 kB 59.7 MB/s 
     |████████████████████████████████| 370 kB 71.6 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=c30cd841eee16a24524e98ec3aa69709333be446923726e433d85b08a0028d57
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting

     |████████████████████████████████| 745 kB 8.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=f86c6b2322af067ade64bbffc20083191dc4782b68d7a81eb1ad292d0b4af61a
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [22]:
%%writefile item.py
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import random
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

from google.colab import drive
drive.mount('/content/drive')
data_read = pd.read_csv('/content/drive/My Drive/LHL_Final_Project/Written_Reviews/Beer_Recommendations_EDA_V01.csv')
# User 10% of the dataset for testing
df_item_based = data_read.sample(frac=0.10, random_state=42)


import streamlit as st
st.set_page_config(layout="wide") 
st.header('Beer Recommendation App - Collaboration Filter')
st.write('This app was built to help you expand your beer horizons')
st.write('Data obtained from https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews')

beer1 = 'Molson Canadian Lager'
beer2 = 'Guinness Extra Stout Original'
beer3 = 'Blue Moon Belgian White'
beer4 = '90 Minute IPA'
beer5 = 'Nut Brown Ale'
beer6 = 'Smoked Porter'
beer7 = 'Gose'



# User input for new beer
st.title('Rate these beer on a scale of 0-5. If you have not tried one, rate how interested you are in trying it:')
with st.form(key='form1'):
  st.write('Molson Canadian Lager')
  rating1 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='1')
  st.write('Guinness Extra Stout Original')
  rating2 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='2')
  st.write('Blue Moon Belgian White')
  rating3 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='3')
  st.write('90 Minute IPA')
  rating4 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='4')
  st.write('Nut Brown Ale')
  rating5 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='5')
  st.write('Smoked Porter')
  rating6 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='6')
  st.write('Gose')
  rating7 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='7')
  
  submit = st.form_submit_button(label = 'Press Here for New Beer!')

text = ['the perfect beer for anyone who loves the summer months hanging with friends on patios and pairing anything barbecue with refreshing brews',
        'the perfect beer for anyone who loves the bitter flavour of black coffee or a strong grapefruit',
        'the perfect beer for anyone who loves coffee-crisp purposely burnt marshmallows and other bold rich flavours',
        'the perfect beer for anyone who loves dried fruit Werther’s Original candy bold flavours and just a pinch of sweet flavours',
        'the perfect beer for anyone who loves scotch campfires and smoked meats smoky brews can be enjoyed all year round',
        'the perfect beer for anyone whos morning routine consists of chugging a glass of orange juice or anyone who never says when when the waiter is adding pepper to your meal',
        'the perfect beers for anyone who drinks green tea in the morning instead of coffee puts cucumber in their water to give it flavour or is semi-addicted to sour penny candies']

user = [(beer1,rating1),(beer2,rating2),(beer3,rating3),(beer4,rating4),(beer5,rating5),(beer6,rating6),(beer7,rating7)]
user = pd.DataFrame(user, columns = ['beer_name', 'user_overall_score'])

user['brewery_name'] =  pd.np.where(user.beer_name.str.contains("Canadian"), 'Molson Coors Canada',
                        pd.np.where(user.beer_name.str.contains("Guinness"), 'Guinness Ltd',
                        pd.np.where(user.beer_name.str.contains("Blue Moon"), 'Coors Brewing Company MolsonCoors',
                        pd.np.where(user.beer_name.str.contains("90 Minute"), 'Dogfish Head Craft Brewery',
                        pd.np.where(user.beer_name.str.contains("Nut Brown Ale"), 'Samuel Smith Old Brewery Tadcaster',
                        pd.np.where(user.beer_name.str.contains("Smoked Porter"), 'Alaskan Brewing Co Alaskan',
                        pd.np.where(user.beer_name.str.contains("Flemish"), 'Brouwerij Van Steenberge NV', 'Westbrook Brewing Co')))))))

user['beer_id'] = pd.np.where(user.beer_name.str.contains("Canadian"), '1312.0',
                  pd.np.where(user.beer_name.str.contains("Guinness"), '650.0',
                  pd.np.where(user.beer_name.str.contains("Blue Moon"), '1212.0',
                  pd.np.where(user.beer_name.str.contains("90 Minute"), '2093.0',
                  pd.np.where(user.beer_name.str.contains("Nut Brown Ale"), '576.0',
                  pd.np.where(user.beer_name.str.contains("Smoked Porter"), '90.0',
                  pd.np.where(user.beer_name.str.contains("Flemish"), '10482.0', '133043.0')))))))

user['text'] = pd.np.where(user.beer_name.str.contains("Lager"), text[0],
               pd.np.where(user.beer_name.str.contains("IPA"), text[1],
               pd.np.where(user.beer_name.str.contains("Stout"), text[2],
               pd.np.where(user.beer_name.str.contains("Brown Ale"), text[3],
               pd.np.where(user.beer_name.str.contains("Smoke"), text[4],
               pd.np.where(user.beer_name.str.contains("White"), text[5], text[6]))))))

new_ratings_df = df_item_based.append(user,ignore_index=True)
new_ratings_df = new_ratings_df.fillna(0)
new_ratings_df = new_ratings_df[['username','Name','user_overall_score']]
new_ratings_df['user_overall_score'] = new_ratings_df['user_overall_score'].astype(float, errors = 'raise')
ratings = new_ratings_df.pivot_table(index='username', columns='Name', values='user_overall_score')
ratings.fillna(0,inplace=True)

# Removing sparsity 
csr_ratings = csr_matrix(ratings.values)
# Standardize the mean of each user to 0, and correct for harsh/lenient users
def standardize(row):
  new_row = (row - row.mean()) / (row.max() - row.min())
  return new_row

ratings_std = ratings.apply(standardize)

# For item-to-item Similarity
item_similarity = cosine_similarity(ratings_std.T)
item_similarity_df = pd.DataFrame(item_similarity,index=ratings_std.columns, columns=ratings_std.columns)



def get_similar_beer(beer_name,user_rating):
  similar_score = item_similarity_df[beer_name]*(user_rating - 2.5)
  similar_score = similar_score.sort_values(ascending = False)
  return similar_score

# Using user input beer
user['Name'] = user['brewery_name']+' '+user['beer_name']
new_user = user[['Name', 'user_overall_score']]
new_user_list = new_user.to_records(index=False).tolist()

similar_beer = pd.DataFrame()

for beer, rating in new_user_list:
  similar_beer = similar_beer.append(get_similar_beer(beer,rating))
  x = similar_beer.sum().sort_values(ascending= False)
  x = pd.DataFrame(x).reset_index()
  x = x.rename(columns={'index': 'Name',0:'Score'})
  y = pd.DataFrame(new_user_list)
  y = y.rename(columns={0: 'Name',1:'Score'})
  x = x[~x['Name'].isin(y['Name'])]
  
# top_choices = x.head(10)
# bottom_choices = x.tail(1)
st.write('Top Beer Choices')
st.write(x['Name'].head(10))
st.write('Or Maybe Something Completely Different')
st.write(x['Name'].tail(1))

Overwriting item.py


In [23]:
!ls

drive  item.py	sample_data


# Insert ngrok authtoken

In [24]:
!ngrok authtoken XXXXXXXXXXXXXXXXXXXXXXXXX
from pyngrok import ngrok
!streamlit run --server.port 80 item.py&>/dev/null&
#publ_url = ngrok.connect(port='8501')
publ_url = ngrok.connect(port='80')
publ_url 

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


<NgrokTunnel: "http://3a0f-34-125-220-192.ngrok.io" -> "http://localhost:80">

In [25]:
ngrok.kill()